In [ ]:

import requests
import threading
import pandas
from bs4 import BeautifulSoup

info = [[] for i in range(100)]


def crawler():
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0',
              'Referer': 'https://maoyan.com/board/4',
              'Cookie': '__mta=156203908.1635846170853.1635998046160.1635998090944.69; uuid_n_v=v1; '
                        'uuid=3D0DE3503BC111EC8618ADF37BAA1C04E1A5FD85F6844214874BC852DD7E592B; '
                        'Hm_lvt_703e94591e87be68cc8da0da7cbd0be2=1635871099,1635872390,1635880319,1635998045; '
                        '_lxsdk_cuid=17ce007a8acc8-0848506244ffee8-4c3e2679-1fa400-17ce007a8acc8; '
                        '_lxsdk=3D0DE3503BC111EC8618ADF37BAA1C04E1A5FD85F6844214874BC852DD7E592B; '
                        '_csrf=0dd866aba775da199e9c5f1ab9b93a3a9c2069f8dcc1ac79f02ac38171768611; '
                        'Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2=1635998091; _lxsdk_s=17ce91516cf-4d6-957-6be%7C%7C4'}
    titles = []
    directors = []
    actors = []
    ratings = []
    incomes = []
    countries = []
    durations = []
    types = []
    years = []
    mutex = threading.Lock()

    def crawl():
        for s in range(0, 100, 10):
            html = requests.get("https://maoyan.com/board/4", params={'offset': s}, headers=header)
            bs = BeautifulSoup(html.text, 'html.parser')
            # Title
            title = bs.select("#app > div > div > div.main > dl > dd > div > div > div > p.name > a")
            for t in title:
                titles.append(t.get_text())
            # Name of actors
            actor = bs.select("#app > div > div > div.main > dl > dd > div > div > div.movie-item-info > p.star")
            for a in actor:
                actors.append(a.get_text()[20:-9].replace(",", "、"))
            # Rating
            rating1 = bs.select(
                "#app > div > div > div.main > dl > dd > div > div > div.movie-item-number.score-num > p > i.integer")
            rating2 = bs.select(
                "#app > div > div > div.main > dl > dd > div > div > div.movie-item-number.score-num > p > i.fraction")
            for r1, r2 in zip(rating1, rating2):
                ratings.append(r1.get_text() + r2.get_text())
            # Year
            year = bs.select(
                "#app > div > div > div.main > dl > dd > div > div > div.movie-item-info > p.releasetime")
            for y in year:
                years.append(y.get_text()[5:15])
            url = bs.select("#app > div > div > div.main > dl > dd > div > div > div.movie-item-info > p.name > a")
            # Get details in movies introduction pages.
            for u in url:
                urls = "https://maoyan.com" + u['href']
                html0 = requests.get(urls, headers=header).text
                soup0 = BeautifulSoup(html0, 'html.parser')
                # Name of director
                director = soup0.select(
                    "#app > div > div.main-content > div > div.tab-content-container > "
                    "div.tab-desc.tab-content.active > "
                    "div:nth-of-type(2) > div.mod-content > div > div:nth-of-type(1) > ul > li > div > a")
                for d in director:
                    directors.append(d.get_text()[7:-5])
                # Cumulative income
                income0 = soup0.select(
                    "#app > div > div.main-content > div > div.tab-content-container > "
                    "div.tab-desc.tab-content.active > "
                    "div:nth-of-type(8) > div.mod-content > div > div:nth-of-type(2)> div")
                if (len(income0) == 0):
                    incomes.append('暂无')
                else:
                    incomes.append(income0[0].get_text())
                # Duration
                country_duration = soup0.select(
                    "body > div.banner > div > div.celeInfo-right.clearfix > div.movie-brief-container > ul > "
                    "li:nth-of-type(2)")
                for c in country_duration:
                    countries.append(c.get_text().partition('/')[0].strip())
                    durations.append(c.get_text().partition('/')[2].strip())
                # Type
                type0_ = soup0.select(
                    "body > div.banner > div > div.celeInfo-right.clearfix > div.movie-brief-container > ul > "
                    "li:nth-of-type(1) > a")
                type0 = ''
                for t in type0_:
                    type0 += t.get_text() + " "
                types.append(type0)
            print("One page of movies has been crawled.")

    def read(i, list):
        mutex.acquire()
        info[i].append(list[i])
        mutex.release()

    print("Crawler is working, please wait for a moment...")
    crawl()
    # Store data in the CSV file
    print("Crawl Accomplished, Start Loading...")
    try:
        for i in range(100):
            read(i, titles)
            read(i, directors)
            read(i, actors)
            read(i, ratings)
            read(i, incomes)
            read(i, countries)
            read(i, durations)
            read(i, types)
            read(i, years)
        index = [i + 1 for i in range(100)]
        name = ['Title', 'Director', 'Actors', 'Rating', 'Income', 'Country', 'Duration', 'Type', 'Year']
        table = pandas.DataFrame(columns=name, index=index, data=info)
        print(table)
        table.to_csv('Shen_Wang_1824260.csv', mode='a+', encoding='utf-8-sig')
        print("Top 100 Movies in MaoYan have been crawled successfully.")
    except IndexError:
        print("Crawl Incomplete, Please Check the Website and Try Again.")
        pass


if __name__ == "__main__":
    crawler()




In [ ]:

import matplotlib.pyplot as plt
import pandas
from collections import Counter
%matplotlib inline

ranking = []
rating = []
income1 = []
income2 = []

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
pandas.set_option('display.width', 500)
pandas.set_option('display.max_columns', 1000)
pandas.set_option('display.notebook_repr_html', True)
table = pandas.read_csv("Shen_Wang_1824260.csv")
table = table.rename(columns={"Unnamed: 0": "Ranking"})


def data_clean():
    for i in table["Income"]:
        if i == "暂无":
            i = 0
            income1.append(i)
        else:
            income1.append(int(i))
    for r, i in zip(table["Ranking"], income1):
        if i != 0:
            ranking.append(r)
    for r, i in zip(table["Rating"], income1):
        if i != 0:
            rating.append(float(r))
            income2.append(i)


# Count the directors with more films on the list
def directors_more_movies():
    director_counter = dict(Counter(table["Director"]))
    directors = []
    times = []
    for key, value in director_counter.items():
        if value > 1:
            directors.append(key)
            times.append(value)
    plt.figure(figsize=(20, 6))
    plt.bar(directors, times, width=0.5, align="center", label="Number of Movies")
    plt.title("Directors with More Movies", loc="center")
    for a, b in zip(directors, times):
        plt.text(a, b, b, ha='center', va='center', fontsize=12)
    plt.ylabel("Movies")
    plt.xlabel("Director")
    plt.legend()
    plt.savefig("1.jpg")
    plt.show()


# The relationship between Ranking and Rating
def ranking_rating():
    plt.figure(figsize=(20, 6))
    x = table["Ranking"]
    y = table["Rating"]
    plt.plot(x, y, color="k", linestyle="dashdot", linewidth=1, label="Rating")
    plt.title("The Relationship between Ranking and Rating", loc="center")
    for a, b in zip(x, y):
        plt.text(a, b, b, ha='center', va="bottom", fontsize=8)
    plt.grid(True)
    plt.legend()
    plt.savefig("2.jpg")
    plt.show()


# The relationship between Ranking and Income
def ranking_income():
    plt.figure(figsize=(20, 6))
    x = ranking
    y = income2
    plt.plot(x, y, color="k", linestyle="dashdot", linewidth=1, label="Income")
    plt.title("The Relationship between Ranking and Income", loc="center")
    for a, b in zip(x, y):
        plt.text(a, b, b, ha='center', va="bottom", fontsize=8)
    plt.grid(True)
    plt.legend()
    plt.savefig("3.jpg")
    plt.show()


# The relationship between Rating and Income
def rating_income():
    plt.figure(figsize=(10, 15))
    x = rating
    y = income2
    plt.scatter(x, y, marker="o", s=5)
    plt.title("The Relationship between Rating and Income", loc="center")
    plt.xlabel("Rating")
    plt.ylabel("Income (*10,000)")
    plt.grid(False)
    plt.savefig("4.jpg")
    plt.show()


# Some analyse about the country
# Country and movie numbers
def country_number():
    country_counter = dict(Counter(table["Country"]))
    countries = []
    times = []
    for key, value in country_counter.items():
        if value > 1:
            countries.append(key)
            times.append(value)
    plt.figure(figsize=(20, 6))
    plt.bar(countries, times, width=0.5, align="center", label="Number of Movies")
    plt.title("Countries with More Movies", loc="center")
    for a, b in zip(countries, times):
        plt.text(a, b, b, ha='center', va='center', fontsize=12)
    plt.ylabel("Movies")
    plt.xlabel("Countries")
    plt.legend()
    plt.savefig("country1.jpg")
    plt.show()


# Country and incomes
def country_incomes():
    countries = []
    for c, i in zip(table["Country"], income1):
        if i != 0:
            countries.append(c)
    plt.figure(figsize=(25, 20))
    x = countries
    y = income2
    plt.scatter(x, y, marker="o", s=10)
    plt.title("Countries and Movies Incomes", loc="center")
    plt.xlabel("Movies Incomes")
    plt.ylabel("Income (*10,000)")
    plt.grid(False)
    plt.savefig("country2.jpg")
    plt.show()


if __name__ == "__main__":
    data_clean()
    directors_more_movies()
    ranking_rating()
    ranking_income()
    rating_income()
    country_number()
    country_incomes()


